In [1]:
from pathlib import Path
import cv2
import sys
import numpy as np
from skimage import feature, measure
import livecell_tracker
from livecell_tracker.core import datasets
from livecell_tracker.core.datasets import LiveCellImageDataset
import livecell_tracker.segment
from livecell_tracker import core
import livecell_tracker.core.utils
from tqdm import tqdm
import json
from livecell_tracker.core import (
    SingleCellTrajectory,
    SingleCellStatic,
    SingleCellTrajectoryCollection,
)
import json

seg_dir_path = r"""../datasets/test_data/track_sort/seg_output_3_imgs_day0_group1"""
label_seg_imgs = core.datasets.LiveCellImageDataset(seg_dir_path, ext="png")

label_seg_imgs = LiveCellImageDataset(seg_dir_path, ext="png", num_imgs=3)
dataset_dir_path = Path(
    r"../cxa-data/june_2022_data_8bit_png/restart_day0_Group 1_wellA1_RI_MIP_stitched"
)

raw_imgs = LiveCellImageDataset(dataset_dir_path, ext="png", num_imgs=3)
segmentation_result_json_path = r"../datasets/test_data/track_sort/seg_output_3_imgs_day0_group1/segmentation_results.json"
segmentation_results = json.load(open(segmentation_result_json_path, "r"))
segmentation_results;

In [2]:
import napari
from skimage import data
# viewer = napari.view_image(raw_imgs[0], name='image')
viewer = napari.view_image(raw_imgs.to_dask(), name='image', cache=True)
# all_imgs = LiveCellImageDataset(dataset_dir_path, ext="png", num_imgs=none)

# viewer = napari.view_image(all_imgs.to_dask(), name='all-image', cache=False)

In [3]:
traj_collection_json_path = "../datasets/test_data/traj_analysis/track_singleCellTrajectoryCollection.json"
traj_collection_json = json.load(open(traj_collection_json_path, "r"))
trajectory_collection = SingleCellTrajectoryCollection().load_from_json_dict(traj_collection_json)

In [4]:
traj_1 = trajectory_collection[1]
time2shape = traj_1.get_sc_napari_shapes()
print(time2shape)
for time, shape in time2shape.items():
    coords = np.array(time2shape[time])[:, 1:]
    time2shape[time] = [[time] + list(coord + time * 100) for coord in coords]
print(time2shape)
rec_shapes = np.array([shape for time, shape in time2shape.items()])
print(rec_shapes)
print(rec_shapes.shape, raw_imgs.to_dask().shape)
shapes_layer = viewer.add_shapes(rec_shapes, name="traj1_slice")
# for traj in trajectory_collection:
#     viewer.add_shapes(traj.get_sc_napari_shapes(), opacity=0.1, name="traj")

{0: [[0, 1294.5, 1164.5], [0, 1294.5, 1477.5], [0, 1567.5, 1477.5], [0, 1567.5, 1164.5]], 1: [[1, 1295.6817736850767, 1161.2969654515127], [1, 1295.6817736850767, 1478.7032343087749], [1, 1573.3175271539167, 1478.7032343087749], [1, 1573.3175271539167, 1161.2969654515127]], 2: [[2, 1296.2285522856919, 1165.9755290801193], [2, 1296.2285522856919, 1476.7303870176272], [2, 1571.300538944855, 1476.7303870176272], [2, 1571.300538944855, 1165.9755290801193]]}
{0: [[0, 1294.5, 1164.5], [0, 1294.5, 1477.5], [0, 1567.5, 1477.5], [0, 1567.5, 1164.5]], 1: [[1, 1395.6817736850767, 1261.2969654515127], [1, 1395.6817736850767, 1578.7032343087749], [1, 1673.3175271539167, 1578.7032343087749], [1, 1673.3175271539167, 1261.2969654515127]], 2: [[2, 1496.2285522856919, 1365.9755290801193], [2, 1496.2285522856919, 1676.7303870176272], [2, 1771.300538944855, 1676.7303870176272], [2, 1771.300538944855, 1365.9755290801193]]}
[[[0.00000000e+00 1.29450000e+03 1.16450000e+03]
  [0.00000000e+00 1.29450000e+03 1.

In [5]:
shapes_layer.data = []

In [6]:
def set_shapes_features(shapes_layer, step2shapes, step):
    # step is a 4D coordinate with the current slider position for each dim
    column = step[0]  # grab the leading ("time") coordinate
    new_data = []
    print(step)
    if step in step2shapes:
        new_data = step2shapes[step]
    shapes_layer.data = new_data
    shapes_layer.face_color = 'value'  # force features refresh


# viewer.dims.events.current_step.connect(
#         lambda event: set_shapes_features(shapes_layer, traj_1.get_sc_napari_shapes(), event.value)
#         )
